In [0]:
import os
import torch
import torchvision
import torch.utils.data as utils
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset
from torch.autograd import Variable
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import timeit
import random

import matplotlib.pyplot as plt
import numpy as np

def imageBatch(nb_image):
    imgBatchR_T = torch.randint(0,255,(nb_image,3,9,9))
    imgBatchL_T = imgBatchR_T
    labelT = torch.ones(nb_image,1)

    imgBatchR_F = torch.randint(0,255,(nb_image,3,9,9)) #
    imgBatchL_F = torch.randint(0,255,(nb_image,3,9,9))
    labelF = torch.zeros(nb_image,1)

    finalR = torch.cat((imgBatchR_T,imgBatchR_F))
    finalL = torch.cat((imgBatchL_T,imgBatchL_F))
    finalLabel = torch.cat((labelT,labelF))

    return finalR, finalL, finalLabel

def firstStageCNN():
    return nn.Sequential(#nn.Conv2d(3,8,kernel_size=2),
                         #nn.ReLU(inplace=True),
                         #Flatten(),
                         
                         nn.Linear(3, 50), #L1
                         nn.ReLU(inplace=True),
                         
                         nn.Linear(50, 50), #L2
                         nn.ReLU(inplace=True))

class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()

        self.cnn1 = firstStageCNN()
        
        self.cnn2 = firstStageCNN()

        self.fc = nn.Sequential(nn.Linear(100, 75), #L3
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(75, 50), #L4
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(50, 25), #L5
                                nn.ReLU(inplace=True),
                                
                                nn.Linear(25, 1)) #L6
    
    def forward(self, input1, input2):
        
        output1 = self.cnn1(input1.float())
        output2 = self.cnn2(input2.float())

        combined = torch.cat((output1.view(output1.size(0), -1),
                              output2.view(output2.size(0), -1)), dim=1)

        combined = torch.unsqueeze(combined,2)
        combined = torch.unsqueeze(combined,3)
        out = self.fc(combined)

        return out

def train(net, finalR, finalL, finalLabel, EPOCHS, BATCH_SIZE):
    optimizer = optim.Adam(net.parameters(), lr=0.1)
    loss_function = nn.MSELoss()
    finalR, finalL, finalLabel = imageBatch(BATCH_SIZE)
    dataset = utils.TensorDataset(finalR, finalL, finalLabel)
    train_dataloader = DataLoader(dataset, shuffle=True, num_workers=0, batch_size=1)
    net.zero_grad()

    print("train function was executed")
    for epoch in range(EPOCHS):
        COUNTER = 0
        for i, data in enumerate(train_dataloader):

            img1, img2, label = data
            optimizer.zero_grad() # reset gradient
            outputs = net(img1, img2)
            loss = loss_function(outputs, label)
            loss.backward()
            optimizer.step()
        #Print out images and epoch numbers 
        print("Epoch number: ", COUNTER)
        COUNTER += 1
        print("Loss:", loss)
        plt.figure()
        plt.imshow((outputs.view(height,width)).detach().numpy())
        plt.show()
        plt.figure()
        plt.imshow((depthmap.view(height,width)).detach().numpy())
        plt.show()
    return net

In [135]:
net = SiameseNetwork()
NumberIMG = 10

#This will import the real dataset in tensor arrays once the data is available
training_DATA_LEFT = imageBatch(NumberIMG)
training_DATA_RIGHT = imageBatch(NumberIMG)
depthM = depthMaps(NumberIMG)

final = train(net,0,0,0,4,10)

train function was executed


RuntimeError: ignored

In [87]:
a,b,c = imageBatch(5)

print(a)
print(b)
print(c)

tensor([[[[139,  75, 154,  ..., 226, 131, 140],
          [180, 166,  16,  ...,  28,  76, 188],
          [174, 141, 235,  ..., 169,  60,  25],
          ...,
          [149,  53, 217,  ...,  48,  86,  59],
          [137, 161,   5,  ..., 121, 189, 217],
          [183,  26, 189,  ..., 102, 133, 244]],

         [[218,  17, 122,  ..., 253,  82, 165],
          [172, 223,  64,  ...,  38, 131,  51],
          [ 54, 133, 156,  ...,   9, 217,  72],
          ...,
          [ 99, 178, 250,  ..., 229,  96, 138],
          [157,  45, 188,  ...,  25, 175,  73],
          [ 36,  56,  73,  ...,  76, 222, 254]],

         [[196,   5, 243,  ..., 152, 218, 161],
          [223, 127, 202,  ..., 133,   5, 223],
          [223,  60,  99,  ..., 116, 160,   6],
          ...,
          [198, 208,  60,  ..., 202,  45,  62],
          [211, 110,  68,  ...,  41, 132, 244],
          [ 77, 168,  88,  ...,  28, 154,  81]]],


        [[[145, 234,  40,  ...,  95, 172, 145],
          [222, 210, 197,  ...,  27